## Exercise 05. Pandas optimizations

1. Считываю `fines.csv` файл, который я сохранял в предыдущем упражнении.

In [1]:
import pandas as pd
import gc

df = pd.read_csv("../data/fines.csv")
df.head()


,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2.0,3200.0,Ford,Focus,1989
1,E432XX77RUS,1.0,6500.0,Toyota,Camry,1995
2,7184TT36RUS,1.0,2100.0,Ford,Focus,1984
3,X582HE161RUS,2.0,2000.0,Ford,Focus,2015
4,92918M178RUS,1.0,5700.0,Ford,Focus,2014


2. Итерации: во всех последующих подзадачах мне нужно будет вычислить `fines/refund*year` для каждой строки. Создаю новый столбец с вычисленными данными. Измеряю время с помощью магической команды `%%timeit` в ячейке.

In [2]:
%%timeit
result = []
for i in range(0, len(df)):
    result.append(df.iloc[i]["Fines"] / df.iloc[i]["Refund"] * df.iloc[i]["Year"])

134 ms ± 21.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
%%timeit
result = []
for idx, row in df.iterrows():
    result.append(row["Fines"] / row["Refund"] * row["Year"])


42.6 ms ± 4.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%%timeit
df.apply(lambda row: row["Fines"] / row["Refund"] * row["Year"], axis=1)


9.81 ms ± 2.13 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%%timeit
df["Fines"] / df["Refund"] * df["Year"]


157 µs ± 27.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
%%timeit
df["Fines"].values / df["Refund"].values * df["Year"].values


12.8 µs ± 978 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


3. Индексирование: измеряю время с помощью магической команды `%%timeit` в ячейке.

In [7]:
%%timeit
df[df["CarNumber"] == "O136HO197RUS"]


267 µs ± 5.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
df_indexed = df.set_index("CarNumber")


In [9]:

%%timeit
df_indexed.loc["O136HO197RUS"]

70.4 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


4. Понижающий коэффициент:

In [10]:
df.info(memory_usage="deep")


optimized_df = df.copy()


for i in optimized_df.select_dtypes(include=["float64"]).columns:
    optimized_df[i] = optimized_df[i].astype("float32")


for i in optimized_df.select_dtypes(include=["int64"]).columns:
    optimized_df[i] = pd.to_numeric(optimized_df[i], downcast="integer")

optimized_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CarNumber  930 non-null    object 
 1   Refund     930 non-null    float64
 2   Fines      930 non-null    float64
 3   Make       930 non-null    object 
 4   Model      919 non-null    object 
 5   Year       930 non-null    int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 174.9 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CarNumber  930 non-null    object 
 1   Refund     930 non-null    float32
 2   Fines      930 non-null    float32
 3   Make       930 non-null    object 
 4   Model      919 non-null    object 
 5   Year       930 non-null    int16  
dtypes: float32(2), int16(1), object(3)
memory usage: 162.1 KB


5. Категории:

In [11]:
for i in optimized_df.select_dtypes(include=["object"]).columns:
    optimized_df[i] = optimized_df[i].astype("category")

optimized_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   CarNumber  930 non-null    category
 1   Refund     930 non-null    float32 
 2   Fines      930 non-null    float32 
 3   Make       930 non-null    category
 4   Model      919 non-null    category
 5   Year       930 non-null    int16   
dtypes: category(3), float32(2), int16(1)
memory usage: 62.5 KB


6. Очистка памяти:

In [12]:
del df
gc.collect()

0

In [13]:
%reset_selective -f df